In [3]:
import sys
from pathlib import Path

In [4]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [7]:
from api.events.model import EventModel
from api.db.session import engine
from sqlmodel import Session, select
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint

In [10]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    # results = session.exec(query).all()
    print(compiled_query)
    print(str(query))

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at
 LIMIT 10
SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at
 LIMIT :param_1


In [19]:
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 minute", EventModel.time)
    pages = ['/about', '/contact', '/pages', '/pricing', 'pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)

    query = (
        select(bucket, EventModel.page, func.count())
        .where(EventModel.time > start, EventModel.time <= finish, EventModel.page.in_(pages))
        .group_by(bucket, EventModel.page)
        .order_by(bucket, EventModel.page)
    )
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    results = session.exec(query).fetchall()
    # pprint(results)
    print(compiled_query)
    # print(str(query))

SELECT time_bucket('1 minute'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS count_1 
FROM eventmodel 
WHERE eventmodel.time > '2025-05-14 03:04:31.810035+00:00' AND eventmodel.time <= '2025-05-14 05:04:31.810048+00:00' AND eventmodel.page IN ('/about', '/contact', '/pages', '/pricing', 'pricing') GROUP BY time_bucket('1 minute'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 minute'::interval, eventmodel.time), eventmodel.page
